In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mode

import neurokit2 as nk

In [2]:
data = pd.read_pickle(f'filt_dataset.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'filt_dataset.pkl'

In [ ]:
def SI(rr_int_ms, window=344, si_percent=0.1):
    """
    
    """
    rr_int_s = rr_int_ms / 1000
    ret = []

    for i in range(len(rr_int_s)-window):
        sample = rr_int_s[i:i+window]
        MO = np.median(sample)
        AMO = (np.abs(sample - MO)<si_percent).mean()*100
        MxDMn = sample.max() - sample.min()
        
        si = AMO/(2*MO*MxDMn)
        if si > 1000:
            sample = sample + np.random.randint(-100, 110, len(sample))/1000
            MO = np.median(sample)
            AMO = (np.abs(sample - MO)<si_percent).mean()*100
            MxDMn = sample.max() - sample.min()
            if (MO*MxDMn) == 0:
                si = 0
            si = AMO/(2*MO*MxDMn)
        
        ret.append(si)
    return np.array(ret)

In [ ]:
person_5_stress_PPG_signal = data['S15']['PPG_medit']

peaks, info = nk.ppg_peaks(person_5_stress_PPG_signal, sampling_rate=64)

peaks_sample_num = []
for i in range(0, len(peaks['PPG_Peaks'])):
    if(peaks['PPG_Peaks'][i] == 1):
        peaks_sample_num.append(i)

r_peaks_seconds = []
for peak in peaks_sample_num:
    r_peaks_seconds.append(peak / 64)

rr_intervals_ms = np.diff(r_peaks_seconds) * 1000

print(SI(rr_intervals_ms)) # значение 50-150 норма, остальное отклонение